In [1]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    ResNet34, EfficientNetV2, build_efficientnetv2,
    Var_ECAPA, MHA_LAS
)
from src.coatnet import CoAtNet
from src.resnetse import ResNetSE, SEBasicBlock

seed_everything(42, workers=True)

Global seed set to 42


42

FIX SPEC AUGMENT

In [2]:
num_classes = 8
dm = VoxCelebDataModule(
    batch_size=16,
    from_memory=True,
    spec_augment=False
    # data_dir="D:\Datasets\subset"
)

In [4]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=aams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type                    | Params
--------------------------------------------------------
0  | loss_func  | AAMSoftmaxLoss          | 2.0 K 
1  | acc        | Accuracy                | 0     
2  | f1         | F1Score                 | 0     
3  | conv1      | Conv2d                  | 3.1 K 
4  | bn1        | BatchNorm2d             | 128   
5  | relu       | ReLU                    | 0     
6  | maxpool    | MaxPool2d               | 0     
7  | conv2_x    | Sequential              | 221 K 
8  | conv3_x    | Sequential           

Epoch 31:  31%|███       | 74/240 [00:06<00:14, 11.81it/s, loss=0.0971, v_num=25, train_loss_step=0.385, train_acc_step=0.938, train_f1_step=0.965, val_loss_step=0.711, val_acc_step=0.909, val_f1_step=0.949, val_loss_epoch=1.420, val_acc_epoch=0.744, val_f1_epoch=0.763, train_loss_epoch=0.0998, train_acc_epoch=0.971, train_f1_epoch=0.971]   

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Epoch 31:  31%|███       | 74/240 [00:20<00:45,  3.65it/s, loss=0.0971, v_num=25, train_loss_step=0.385, train_acc_step=0.938, train_f1_step=0.965, val_loss_step=0.711, val_acc_step=0.909, val_f1_step=0.949, val_loss_epoch=1.420, val_acc_epoch=0.744, val_f1_epoch=0.763, train_loss_epoch=0.0998, train_acc_epoch=0.971, train_f1_epoch=0.971]

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm,
    ckpt_path=checkpoint_path
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=30
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger("tb_logs", name="efficientnetv2")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    margin=0.2,
    scale=30
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = MHA_LAS(
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="AdamW",
    lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger("tb_logs", name="mha_las")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = Var_ECAPA(num_classes=5, loss_func=aams)
logger = TensorBoardLogger("tb_logs", name="var_ecapa")
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
from torch import nn

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/coatnet/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = CoAtNet(in_ch=1, embeddings_dim=5, image_size=224, num_classes=5, loss_func=ce_loss)
logger = TensorBoardLogger("tb_logs", name="coatnet")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetse/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [16, 32, 64, 128]
model = ResNetSE(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger("tb_logs", name="resnetse")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)